In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_66Agree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_AllAgree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/README.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/License.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_75Agree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_50Agree.txt


In [1]:
!pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.2 which is incompatible.
cudf 24.4.1 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.5.0 which is incompatible.
distributed 2024.1.1 requires dask==2024.1.1, but you have dask 2024.5.2 which is incompatible.
gc

In [2]:
!pip install -q -U git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e
!pip install -q -U git+https://github.com/huggingface/peft@4a1559582281fc3c9283892caea8ccef1d6f5a4f

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.


In [36]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
warnings.filterwarnings("ignore")

In [37]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [38]:
print(f"pytorch version {torch.__version__}")

pytorch version 2.1.2


In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


In [40]:
df = pd.read_csv('/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv',
                names=["sentiment", "text"],
                 encoding="utf-8", encoding_errors="replace")

In [41]:
df.head(5)

,sentiment,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [42]:
X_train = []
X_test = []
for sentiment in ["positive", "neutral", "negative"]:
    train, test  = train_test_split(df[df.sentiment==sentiment], 
                                    train_size=300,
                                    test_size=300, 
                                    random_state=42)
    X_train.append(train)
    X_test.append(test)
X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)
eval_idx = [idx for idx in df.index if idx not in list(X_train.index) + list(X_test.index)]
# if it dont exist 
X_eval = df[df.index.isin(eval_idx)]
X_eval = (X_eval
          .groupby('sentiment', group_keys=False)
          .apply(lambda x: x.sample(n=50, random_state=10, replace=True)))
X_train = X_train.reset_index(drop=True)

In [43]:
X_train

,sentiment,text
0,neutral,Mr Jortikka is president of the base metal div...
1,positive,Both operating profit and net sales for the 12...
2,negative,Finnish automation solutions developer Cencorp...
3,positive,Renzo Piano 's building design will be a wonde...
4,positive,`` We are proud to contribute to the creation ...
...,...,...
895,neutral,"The dividend will be paid on April 15 , 2008 t..."
896,neutral,The new shares entitle their holders to divide...
897,neutral,Activities range from the development of natur...
898,positive,"According to Bosse , the present cooperation i..."


In [44]:
def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [{data_point["text"]}] = {data_point["sentiment"]}
            """.strip()


def generate_test_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [{data_point["text"]}] = """.strip()

X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), 
                       columns=["text"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1), 
                      columns=["text"])

In [45]:
y_true = X_test.sentiment
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [46]:
def evaluate(y_true, y_pred):
    labels = ['positive', 'neutral', 'negative']
    mapping = {'positive': 2, 'neutral': 1, 'none':1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)
    
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true)) 
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [47]:
model_name = "ahxt/LiteLlama-460M-1T"
#model_name = "NousResearch/Llama-2-7b-hf"
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

In [48]:
for i,v in X_train[:3].iterrows():
    print(v['text'])
    print('xxxxxxxx')

Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [Mr Jortikka is president of the base metal division of Outotec Oyj in Finland and is on the executive committee of Outotec .] = neutral
xxxxxxxx
Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative".

            [Both operating profit and net sales for the 12-month period increased , respectively from EUR21 .5 m and EUR196 .1 m , as compared to 2005 .] = positive
xxxxxxxx
Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
         

In [49]:
prompt = '[Q: What is the largest bird?\nA:]'
tokenizer(prompt, return_tensors="pt").input_ids

tensor([[   58,    48,    25,  1867,   318,   262,  4387,  6512,    30,   198,
            32, 47715]])

In [50]:
for i,v in X_train[:3].iterrows():
    print(tokenizer.tokenize(v['text']))
    print('xxxxxxxx')

['Analy', 'ze', 'Ġthe', 'Ġsentiment', 'Ġof', 'Ġthe', 'Ġnews', 'Ġheadline', 'Ġenclosed', 'Ġin', 'Ġsquare', 'Ġbrackets', ',', 'Ġ', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġdetermine', 'Ġif', 'Ġit', 'Ġis', 'Ġpositive', ',', 'Ġneutral', ',', 'Ġor', 'Ġnegative', ',', 'Ġand', 'Ġreturn', 'Ġthe', 'Ġanswer', 'Ġas', 'Ġ', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġthe', 'Ġcorresponding', 'Ġsentiment', 'Ġlabel', 'Ġ', '"', 'positive', '"', 'Ġor', 'Ġ', '"', 'neutral', '"', 'Ġor', 'Ġ', '"', 'negative', '"', '.', 'ĊĊ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ[', 'Mr', 'ĠJ', 'ort', 'ik', 'ka', 'Ġis', 'Ġpresident', 'Ġof', 'Ġthe', 'Ġbase', 'Ġmetal', 'Ġdivision', 'Ġof', 'ĠOut', 'ot', 'ec', 'ĠOy', 'j', 'Ġin', 'ĠFinland', 'Ġand', 'Ġis', 'Ġon', 'Ġthe', 'Ġexecutive', 'Ġcommittee', 'Ġof', 'ĠOut', 'ot', 'ec', 'Ġ.', ']', 'Ġ=', 'Ġneutral']
xxxxxxxx
['Analy', 'ze', 'Ġthe', 'Ġsentiment', 'Ġof', 'Ġthe', 'Ġnews', 'Ġheadline', 'Ġenclosed', 'Ġin', 'Ġsquare', 'Ġbr

In [51]:
print("PAD token ID:", tokenizer.pad_token_id)  # Should show the ID of the padding token
print("EOS token ID:", tokenizer.eos_token_id)  # Should show the ID of the EOS token
print("PAD token:", tokenizer.pad_token)        # Should show the actual padding token string
print("EOS token:", tokenizer.eos_token) 

PAD token ID: 50257
EOS token ID: 50257
PAD token: <|im_end|>
EOS token: <|im_end|>


In [52]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50259, 1024)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=False)
          (k_proj): Linear4bit(in_features=1024, out_features=128, bias=False)
          (v_proj): Linear4bit(in_features=1024, out_features=128, bias=False)
          (o_proj): Linear4bit(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=1024, out_features=4096, bias=False)
          (up_proj): Linear4bit(in_features=1024, out_features=4096, bias=False)
          (down_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMS

In [53]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens = 1, 
                        temperature = 0.0,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    return y_pred

In [54]:
y_pred = predict(test, model, tokenizer)


100%|██████████| 900/900 [01:08<00:00, 13.05it/s]


In [55]:
evaluate(y_true, y_pred)

Accuracy: 0.333
Accuracy for label 0: 0.000
Accuracy for label 1: 1.000
Accuracy for label 2: 0.000

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       300
           1       0.33      1.00      0.50       300
           2       0.00      0.00      0.00       300

    accuracy                           0.33       900
   macro avg       0.11      0.33      0.17       900
weighted avg       0.11      0.33      0.17       900


Confusion Matrix:
[[  0 300   0]
 [  0 300   0]
 [  0 300   0]]


In [56]:
output_dir="trained_weigths"

peft_config = LoraConfig(
        lora_alpha=16, 
        lora_dropout=0.1,
        r=64,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)


training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=3,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=True,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch"               # save checkpoint every epoch
)


In [57]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [58]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,0.927700,0.795103
2,0.752400,0.750556


TrainOutput(global_step=336, training_loss=0.9340126131262098, metrics={'train_runtime': 1155.1251, 'train_samples_per_second': 2.337, 'train_steps_per_second': 0.291, 'total_flos': 854203480940544.0, 'train_loss': 0.9340126131262098, 'epoch': 2.99})

In [61]:
# Save trained model and tokenizer
# trainer.save_model()
# tokenizer.save_pretrained(output_dir)
new_model = "LiteLlama-460M-1T-sentiment-ana"

trainer.model.save_pretrained(new_model)


In [62]:
# %load_ext tensorboard
# %tensorboard --logdir logs/runs
new_model = "LiteLlama-460M-1T-sentiment-ana"
base_model = "ahxt/LiteLlama-460M-1T"

In [64]:
from peft import PeftModel

In [68]:
#trainer.model.save_pretrained(new_model)
from peft import AutoPeftModelForCausalLM
# Reload model in FP16 and merge it with LoRA weights
# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     torch_dtype=compute_dtype,
#      return_dict=False,
#      low_cpu_mem_usage=True,
#      device_map=device,
# )
# model = PeftModel.from_pretrained(model, new_model)
# model = model.merge_and_unload()

# # Reload tokenizer to save it
# tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"





# from peft import AutoPeftModelForCausalLM

finetuned_model = "./trained_weigths/"
compute_dtype = getattr(torch, "float16")
tokenizer = AutoTokenizer.from_pretrained("ahxt/LiteLlama-460M-1T")

model = AutoPeftModelForCausalLM.from_pretrained(
     finetuned_model,
     torch_dtype=compute_dtype,
     return_dict=False,
     low_cpu_mem_usage=True,
     device_map=device,
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained("./merged_model",safe_serialization=True, max_shard_size="2GB")
tokenizer.save_pretrained("./merged_model")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('./merged_model/tokenizer_config.json',
 './merged_model/special_tokens_map.json',
 './merged_model/vocab.json',
 './merged_model/merges.txt',
 './merged_model/added_tokens.json',
 './merged_model/tokenizer.json')

In [69]:
# import gc

# del [model, tokenizer, peft_config, trainer, train_data, eval_data, bnb_config, training_arguments]
# del [df, X_train, X_eval]
# del [TrainingArguments, SFTTrainer, LoraConfig, BitsAndBytesConfig]
# for _ in range(100):
#     torch.cuda.empty_cache()
#     gc.collect()
y_pred = predict(test, merged_model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 900/900 [00:32<00:00, 27.91it/s]

Accuracy: 0.767
Accuracy for label 0: 0.887
Accuracy for label 1: 0.770
Accuracy for label 2: 0.643

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.89      0.87       300
           1       0.67      0.77      0.72       300
           2       0.79      0.64      0.71       300

    accuracy                           0.77       900
   macro avg       0.77      0.77      0.77       900
weighted avg       0.77      0.77      0.77       900


Confusion Matrix:
[[266  28   6]
 [ 24 231  45]
 [ 22  85 193]]


In [ ]:
### bump in accuracy from 0.33 to 0.77